# This is the master notebook where I can:
- [ ] Generate log files
    - [ ] Initiailze network parameters
        - [ ] .bin files for validation on hardware
        - [ ] .mif files (hex for simulation, text for decimal values)
    - [ ] Run network
        - [ ] run fixed point check -> output in binary, hex and decimal?
        - [ ] run pytorch check? -> output in decimal
        - [ ] Generate log file with python script and c program timing -> most important?, single forward pass, and time for one pass through an epoch without training
    - [ ] Compare python script and c program accuracy with fixed-point accuracy -> compares data from output files
        - [ ]  Generate log file with the results.
- Directory structure?
    - Notebook with variou

# Directory Structure
- Scripts
    - img
        - MNIST_train.txt
        - MNIST_test.txt
        - MNIST_train.bin
        - MNIST_test.bin
    - runs
        - 11.11.2024 -x32-10-10
            - Init parameters and expected output?
                * binary_files
                * mif files (hex?)
                * txt files (Decimal?)
            - Results
                * binary outputs (final parameters in binary?)
                * text outputs (performance? Training results?)
                * 
    - PLNN_master.ipynb
    - Initialize network run.py -> generates a run for the current date

### Design Initialization script
#### this script randomly generates the initiailization weights and biases, and stores them in the relevant files

In [1]:
import numpy as np
from fxpmath import Fxp
import sys
import os
import struct
import glob
from datetime import datetime
from fxpmath import Fxp

DEBUG_MODE = True
INPUT_SIZE = 784

network_parameters = [10,10]

runs_folder = "runs"
now = datetime.now()
date = '.'.join([str(k) for k in [now.month, now.day, now.year]])
run_name = '-'.join((date, '.'.join([str(k) for k in network_parameters])))
run_name = 'test'
run_path = "\\".join((runs_folder, run_name))


bin_folder = "\\".join((run_path,"bin_files")) # should include a file for expected outputs as a lits of output values for all layers concatenated
mif_folder = "\\".join((run_path,"mif_files")) # hex files for simulation go here including expected values
txt_folder = "\\".join((run_path,"txt_files")) # same as the above but in decimal representation
folders = [bin_folder, mif_folder, txt_folder]

x32 = Fxp(-7.25, dtype='S5.27') # For now hardwired to 32/64 bits, but could be made to be generalizable
x64 = Fxp(-7.25, dtype='S10.54')
weights = []
bias = []


if os.path.isdir(run_path) and not DEBUG_MODE:
    raise ValueError(f"The directory \"{"\\".join((runs_folder, run_name))}\", already exists!")

os.makedirs(run_path,exist_ok=True)
for folder in folders:
    os.makedirs(folder,exist_ok=True)
        
# This is where I should start generating random weights and biases and save them to the various folders

for layer in range(len(network_parameters)):
    
    input_size_for_layer = INPUT_SIZE if layer == 0 else network_parameters[layer - 1]
    weights.append((np.random.rand(input_size_for_layer, network_parameters[layer]) * 2) - 1) # This generates uniformly random weights within range {-1, 1}
    bias.append((np.random.rand(network_parameters[layer]) * 2) - 1) # This generates uniformly random biases within range {-1, 1}

    with open(os.path.join(mif_folder, f"bias_{layer}.mif"), "w") as file:
        for i in range(network_parameters[layer]):
            file.write(str(x32(bias[layer][i]).hex()) + "\n")
        
    with open(os.path.join(txt_folder, f"bias_{layer}.txt"), "w") as file:
        for i in range(network_parameters[layer]):
            file.write(str(float(x32(bias[layer][i]))) + "\n")
            
    with open(os.path.join(bin_folder, f"bias_{layer}.bin"), "wb") as file:
        for i in range(network_parameters[layer]):
            file.write(struct.pack('<I', int(x32(bias[layer][i]).hex(),0)))

    for neuron in range(network_parameters[layer]):
                
        with open(os.path.join(mif_folder, f"weight_{layer}_{neuron}.mif"), "w") as file:
            for i in range(input_size_for_layer):
                file.write(str(x32(weights[layer][i, neuron]).hex()) + "\n")
            
        with open(os.path.join(txt_folder, f"weight_{layer}_{neuron}.txt"), "w") as file:
            for i in range(input_size_for_layer):
                file.write(str(float(x32(weights[layer][i, neuron]))) + "\n")
                
        with open(os.path.join(bin_folder, f"weight_{layer}_{neuron}.bin"), "wb") as file:
            for i in range(input_size_for_layer):
                file.write(struct.pack('<I', int(x32(weights[layer][i, neuron]).hex(),0)))


# That should be it for initialization, I can have a seperate script handle the expected results -> start with expected fixed-point values? -> write to the same folders

In [2]:
%run init_design.py [10,10] True

### Fixed-point network simulator
#### Accepts as a parameter the run name

In [13]:
# Thank you Chatgpt!!!
def twos_complement(bin_str):
    """Returns the two's complement of a binary string."""
    # Invert the bits
    inverted = ''.join('1' if bit == '0' else '0' for bit in bin_str)
    # Add 1 to the inverted binary string
    carry = 1
    result = list(inverted)
    
    for i in range(len(inverted) - 1, -1, -1):
        if result[i] == '1' and carry == 1:
            result[i] = '0'
        elif result[i] == '0' and carry == 1:
            result[i] = '1'
            carry = 0
        # No carry to add; break early
        if carry == 0:
            break
    
    return ''.join(result)

def add_binary(bin_str1, bin_str2):
    """Adds two binary strings and returns the result as a binary string, 
    handling overflow by setting the result to the max/min value depending on overflow direction."""
    max_len = max(len(bin_str1), len(bin_str2))
    bin_str1 = bin_str1.zfill(max_len)
    bin_str2 = bin_str2.zfill(max_len)
    
    carry = 0
    result = []
    
    # Perform bitwise addition from LSB to MSB
    for i in range(max_len - 1, -1, -1):
        bit_sum = carry + int(bin_str1[i]) + int(bin_str2[i])
        result.append(str(bit_sum % 2))
        carry = bit_sum // 2
    
    result = ''.join(result[::-1])  # Reverse the result to get the correct order
    
    # Check for overflow
    sign1 = bin_str1[0]  # Sign bit of the first number
    sign2 = bin_str2[0]  # Sign bit of the second number
    result_sign = result[0]  # Sign bit of the result

    # Overflow occurs if both numbers have the same sign but the result has a different sign
    if sign1 == sign2 and sign1 != result_sign:
        if sign1 == '0':  # Positive overflow
            result = '0' + '1' * (max_len - 1)  # Max positive value: 011...111
            # return 'fail'
            raise Exception(f"OVERFLOW")
        else:  # Negative overflow
            result = '1' + '0' * (max_len - 1)  # Max negative value: 100...000
            raise Exception(f"OVERFLOW")
            
    
    return result


def manual_binary_multiply(bin_str1, bin_str2):
    """Multiplies two binary strings in two's complement format manually."""
    # Check for sign and convert to positive if necessary
    is_negative1 = bin_str1[0] == '1'
    is_negative2 = bin_str2[0] == '1'
    
    if is_negative1:
        bin_str1 = twos_complement(bin_str1)
    if is_negative2:
        bin_str2 = twos_complement(bin_str2)

    # Perform binary multiplication (manual)
    len1 = len(bin_str1)
    len2 = len(bin_str2)
    result = '0' * (len1 + len2)
    
    for i in range(len2 - 1, -1, -1):
        if bin_str2[i] == '1':
            # Shift bin_str1 by (len2 - 1 - i) and add to the result
            shifted_bin_str1 = bin_str1 + '0' * (len2 - 1 - i)
            result = add_binary(result, shifted_bin_str1.zfill(len1 + len2))
    
    # Determine the sign of the result
    if is_negative1 != is_negative2:
        result = twos_complement(result.zfill(len1 + len2))

    # Truncate the result to the appropriate length (len1 + len2 bits)
    return result[-(len1 + len2):]


def convert32_64(bin_str1):
    bin_str2 = ['0']*64
    if bin_str1[0] == '1': # this means the number is negative
        bin_str2[:5] = '1'*5
        
    bin_str2[5:len(bin_str1)+5] = bin_str1
    bin_str2 = ''.join(bin_str2)
        
    return bin_str2

def convert64_32(bin_str1):
    if bin_str1[0] == '1':
        if bin_str1[1:5] != '1111':
            print(bin_str1)
            raise Exception(f"OVERFLOW")
        else:
            bin_str2 = '0'*32
    else:
        if bin_str1[1:5] != '0000':
            print(bin_str1)
            raise Exception(f"OVERFLOW")
        else:
            bin_str2 = bin_str1[5:32+5]
        
    return bin_str2

In [126]:
import numpy as np
from fxpmath import Fxp
import sys
import os
import struct
import glob
from datetime import datetime
from fxpmath import Fxp

# script arguments vvv
network_parameters = [10,10]
run_name = 'test'
DEBUG_MODE = True
# script below

runs_folder = "runs"
img_path = "img"
run_path = "\\".join((runs_folder, run_name))
    
if not os.path.isdir(run_path):
    raise ValueError(f"The directory \"{"\\".join((runs_folder, run_name))}\", does not exist!")
    
if not os.path.isdir(img_path):
    raise ValueError(f"The directory \"{img_path}\", does not exist!")

bin_folder = "\\".join((run_path,"bin_files")) # should include a file for expected outputs as a lits of output values for all layers concatenated
mif_folder = "\\".join((run_path,"mif_files")) # hex files for simulation go here including expected values
txt_folder = "\\".join((run_path,"txt_files")) # same as the above but in decimal representation
folders = [bin_folder, mif_folder, txt_folder]

INPUT_SIZE=784

x32 = Fxp(-7.25, dtype='S5.27') # For now hardwired to 32/64 bits, but could be made to be generalizable
x64 = Fxp(-7.25, dtype='S10.54')
weights = []
bias = []

with open(os.path.join(img_path, f"MNIST_train.bin"), 'rb') as file:
    data = file.read()
    format_string = f'{60000*784}I'  # 'I' for unsigned int, repeat for num_elements
    train_img = struct.unpack(format_string, data)
    train_img = np.array(train_img).reshape(60000,784) # formatting image data
    
with open(os.path.join(img_path, f"MNIST_test.bin"), 'rb') as file:
    data = file.read()
    format_string = f'{10000*784}I'  # 'I' for unsigned int, repeat for num_elements
    test_img = struct.unpack(format_string, data)
    test_img = np.array(test_img).reshape(10000,784) # formatting image data


# Reading weights and biases from files for each layer
for layer in range(len(network_parameters)):
    input_size_for_layer = INPUT_SIZE if layer == 0 else network_parameters[layer - 1]
    
    # Initialize weights and bias for the current layer
    weights_layer = np.zeros((input_size_for_layer, network_parameters[layer]),dtype=( np.str_, 10)) #
    bias_layer = np.zeros(network_parameters[layer],dtype=( np.str_, 10))
    
    # Read biases in hexfor the current layer
    with open(os.path.join(mif_folder, f"bias_{layer}.mif"), "r") as file:
        lines = file.readlines()
        for i in range(network_parameters[layer]):
            bias_layer[i] = lines[i].strip()
    bias.append(bias_layer)

    # Read weights for the current layer
    for neuron in range(network_parameters[layer]):
        with open(os.path.join(mif_folder, f"weight_{layer}_{neuron}.mif"), "r") as file:
            lines = file.readlines()
            for i in range(input_size_for_layer):
                weights_layer[i, neuron] = lines[i].strip()
                
    weights.append(weights_layer)

for f_img in train_img:
    img = np.array([x32(k).bin() for k in f_img])
    
    a_tdata = []
    a_tdata_float = []
    for layer in range(len(network_parameters)):
        input_size_for_layer = INPUT_SIZE if layer == 0 else network_parameters[layer - 1]
        input_data = img if layer == 0 else a_tdata[layer - 1]
        layer_output = []
        # layer_output_float = []
        for j in range(network_parameters[layer]):
            b = bias[layer][j]
            acc = convert32_64(x32(b).bin())
            
            for i in range(input_size_for_layer):
                w, x = x32(weights[layer][i, j]).bin(), input_data[i]
                p = manual_binary_multiply(w, x)
                acc = add_binary(acc, p)
    
            
    
            layer_output.append(convert64_32(acc))
            
        a_tdata.append(layer_output)


In [131]:
f_img = train_img[0]

In [132]:
f_img = img/255

In [144]:
img = np.array([x32(k).bin() for k in f_img])
img

array(['00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '00000000000000000000000000000000',
       '000

In [148]:
import time
time.time()

1731382799.8487787

In [149]:
# Feedforward fixed-point calculation for each layer
start = time.time()

a_tdata = []
a_tdata_float = []
for layer in range(len(network_parameters)):
    input_size_for_layer = INPUT_SIZE if layer == 0 else network_parameters[layer - 1]
    input_data = img if layer == 0 else a_tdata[layer - 1]
    layer_output = []
    layer_output_float = []
    for j in range(network_parameters[layer]):
        b = bias[layer][j]
        acc = convert32_64(x32(b).bin())
        
        b_float = float(x32(bias[layer][j]))
        acc_float = b_float

        if  float(x32(b)) != b_float:
            raise Exception(f"BIAS %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", float(x32(b)), b_float)
        
        for i in range(input_size_for_layer):
            w, x = x32(weights[layer][i, j]).bin(), input_data[i]
            p = manual_binary_multiply(w, x)
            # if add_binary(acc,p) == 'fail': 
            #     raise Except(f"ERROR OVERFLOW at {layer, j, i}")
            acc = add_binary(acc, p)
            if len(x32(input_data[i]).bin()) != 32:
                print(i)
                print(input_data[i])
                
            w_float, x_float = float(x32(weights[layer][i, j])), float(x32('0b'+input_data[i]))
            p_float = w_float * x_float
            acc_float = acc_float + p_float

            # print(val1, end='\n')

            val1 = float(x64('0b'+acc))
            val2 = float(x32('0b'+w))
            val3 = float(x32('0b'+x))
            val4 = float(x64('0b'+p))
            
            if  val2 != w_float:
                raise Exception(f"WEIGHTS {val2}, {w_float}, are not equal at layer {layer}, neuron {j} and input value {i}")
            if  val3 != x_float:
                raise Exception(f"img {val3}, {x_float}, are not equal at layer {layer}, neuron {j} and input value {i}")
            if  val4 != p_float:
                raise Exception(f"SUM {val4}, {p_float}, are not equal at layer {layer}, neuron {j} and input value {i}")
            if  abs(val1 - acc_float) > 0.0000001: # This verifies that the fixed point calculation agrees with the floating point calculation within 7 decimal places...
                raise Exception(f" ACCUMULATE {val1}, {acc_float}, are not equal at layer {layer}, neuron {j} and input value {i}")

        

        layer_output.append(convert64_32(acc))
        
        layer_output_float.append(acc_float)
        
    a_tdata.append(layer_output)
    
    a_tdata_float.append(layer_output_float)

end = time.time()
print(end - start)

8.082443475723267


In [ ]:
###### Simplified code without the check

In [150]:
# Feedforward fixed-point calculation for each layer
start = time.time()

a_tdata = []
a_tdata_float = []
for layer in range(len(network_parameters)):
    input_size_for_layer = INPUT_SIZE if layer == 0 else network_parameters[layer - 1]
    input_data = img if layer == 0 else a_tdata[layer - 1]
    layer_output = []
    # layer_output_float = []
    for j in range(network_parameters[layer]):
        b = bias[layer][j]
        acc = convert32_64(x32(b).bin())
        
        for i in range(input_size_for_layer):
            w, x = x32(weights[layer][i, j]).bin(), input_data[i]
            p = manual_binary_multiply(w, x)
            acc = add_binary(acc, p)

        

        layer_output.append(convert64_32(acc))
        
    a_tdata.append(layer_output)

end = time.time()
print(end - start)

2.2318379878997803


In [25]:
import matplotlib.pyplot as plt

In [44]:
weights[0][:,9]

array(['0x047730C2', '0x0416761A', '0xFABE3E99', '0xFC9EA17A',
       '0xFBCB24EB', '0x05D33D14', '0xFF949980', '0x022CFF03',
       '0xFDF39C7B', '0xFB0FDD54', '0xFF5B0115', '0xFB4C5035',
       '0x07A565CD', '0xFF36AEE7', '0x023F3AC3', '0x03C38E1E',
       '0xFBBD1E8C', '0xFCB72510', '0xFA9410D7', '0x04D56A20',
       '0xF99768D8', '0xFAAC5FC3', '0x04CCEA68', '0x0224F24B',
       '0xFA7B2828', '0xFE9485CC', '0xFBB44602', '0x01D0C54F',
       '0x03518D87', '0xFE169511', '0x05D769F6', '0x00D34425',
       '0x01B57594', '0x03A7B9D7', '0xFA7A3057', '0x07A199F0',
       '0x00294738', '0x06ADD4B6', '0x047DF592', '0x023953D4',
       '0x0061D0E4', '0x000BD061', '0xF883076D', '0xFF82B167',
       '0x00397744', '0xFF5C2F52', '0x04957A3F', '0xFA574CB1',
       '0x05656DBB', '0xFA737EBC', '0x010618E4', '0x0628A2E4',
       '0x00AD9FE1', '0x025CBF9C', '0xF977BD30', '0x04FDFB76',
       '0xFC7446B2', '0xFDF8E321', '0x06AE43AB', '0xF88E9E28',
       '0xFC27B1B2', '0x0418F2E8', '0x06A15D6B', '0x016

In [76]:
debug_w = np.zeros(10)
debug_b = np.zeros(10)
for i,k in enumerate(weights[1][:,9]):
    debug_w[i] = (x32(k))
    debug_b[i] = (x32(bias[1][i]))
debug_w, debug_b

(array([ 0.53285456,  0.33818927,  0.22638685,  0.95745411, -0.69228742,
         0.84731783,  0.8728786 ,  0.50675798,  0.68022852,  0.43751545]),
 array([ 0.45902867,  0.1687721 ,  0.52231173,  0.902478  , -0.71121426,
        -0.22792032, -0.24921701,  0.99249978, -0.59037039,  0.12654991]))

In [ ]:
debug_b + debug_w * 

In [19]:
x64('0b1111011110001101010011011000000101011001001000000010011001110001'), 2**5

(fxp-s64/54(-33.792144453967374), 32)

In [65]:
acc

'1111111111100100101011111001110000111000000000000000000000000000'

In [68]:
x64 = Fxp(-7.25, dtype='S10.54')
x64.bin()

'1111111000110000000000000000000000000000000000000000000000000000'

In [71]:
x64('0b'+acc)

fxp-s64/54(-0.426781602203846)

In [64]:
x64(acc)

fxp-s64/54(512.0)

In [53]:
x32('00000000011111101000011111000100')


fxp-s32/27(15.99999999254942)

In [10]:
x32(weights[0][0][0])

fxp-s32/27(0.061782389879226685)

In [14]:
x32(weights[0][0][0]).bin()

'00000000011111101000011111000100'

In [71]:
layer = 0

In [80]:

bin_folder = "\\".join((run_path,"bin_files")) # should include a file for expected outputs as a lits of output values for all layers concatenated
mif_folder = "\\".join((run_path,"mif_files")) # hex files for simulation go here including expected values
txt_folder = "\\".join((run_path,"txt_files")) # same as the above but in decimal representation
folders = [bin_folder, mif_folder, txt_folder]

with open(os.path.join(mif_folder, f"bias_{layer}.mif"), "r") as file:
    lines = file.readlines()
    print(lines[0].strip())

0xFC95F387


In [91]:
debug =  np.zeros((1, 1),dtype=( np.str_, 10))
debug[0]='FFAAFFAA'
debug[0] = lines[0].strip()
debug

array([['0xFC95F387']], dtype='<U10')

In [8]:
with open(os.path.join(img_path, f"MNIST_train.bin"), 'rb') as file:
    IMG_SET_BINARY = file.read()

In [17]:
len(IMG_SET_BINARY)


188160000

In [22]:
IMG_SET_BINARY[:4]

b'\x00\x00\x00\x00'

In [24]:
len(IMG_SET_BINARY.hex())

376320000

In [14]:
376320000/(60000*784)

8.0

In [18]:
188160000/(60000*784)

4.0

In [27]:
x32 = Fxp(-7.25, dtype='S5.27') 

In [28]:
def load_binary_file(filename, num_elements):
    with open(filename, 'rb') as file:
        # Calculate the number of bytes to read (4 bytes per uint32)
        data = file.read(num_elements * 4)
        
        # Unpack the binary data as uint32 (32-bit unsigned integers)
        format_string = f'{num_elements}I'  # 'I' for unsigned int, repeat for num_elements
        values = struct.unpack(format_string, data)
    
    return values

In [36]:
filename = os.path.join(img_path, f"MNIST_train.bin")
num_elements = 60000 * 784  # Total number of uint32 values

# Load the binary data
img_data = load_binary_file(filename, num_elements)


0

In [37]:
img_data = np.array(img_data).reshape(60000,784)
img_data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [ ]:
for i in range(60000):
    for j in range(784):
        